# Model Context Protocol (MCP)

## ¿Qué es MCP?

MCP (Model Context Protocol) es un estandar open source desarrollado por Anthropic para permitir a los modelos de IA interactuar con herramientas externas mediante un estandar

Hasta el desarrollo del protocolo MCP, cuando queríamos que un LLM interactuara con herramientas, teníamos que crear código para poder interactuar con la herramienta, y mediante `function calling` enviarle la información al LLM.

![MCP vs API](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/MCP_vs_APIs.webp)

Así que mediante MCP, un LLM puede interactuar con herramientas gracias a un estandar. De esta manera si una persona crea un servidor MCP, dicho servidor puede ser reutilizado por otros con un único cliente. Si en tu aplicación desarrollas un cliente, puedes descargarte un servidor MCP desarrollado por otro, y usarlo sin problema.

Comunmente MCP se asemeja al estandar USB. Antes del USB, cada periférico tenía un tipo de conexión diferente, unos tenían puertos serie, otros paralelo. Diferentes formatos de conectores, etc.

![USB MCP](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/mcp-usb.webp)

Con la llegada del USB, todos los periféricos se adaptaron a este estandar, por lo que con un solo conector USB en tu ordenador, puedes conectar casi cualquier periférico.

MCP tiene 7 componentes principales:
 * **Host**: Aplicación LLM que tiene acceso a herramientas MCP.
 * **Servidor MCP**: Servidor que realiza la comunicación con la API o herramienta a la que queremos exponer al LLM
 * **Cliente MCP**: Cliente que se conecta al servidor MCP y realiza las peticiones
 * **Tool**: Función que se ejecuta en el servidor MCP y que puede ser invocada por el LLM
 * **Resource**: Recurso que se puede usar en el servidor MCP. Suelen dar al LLM acceso a recursos estáticos como archivos, bases de datos, etc.
 * **Resource template**: Template para crear recursos dinámicos. Mediante estas plantillas, el LLM puede crear dinámicamente el recurso al que quiere acceder
 * **Prompt**: Prompt que se usa para generar un prompt que será usado por el LLM para interactuar con el servidor MCP.











Un único host (aplicación) puede tener varios clientes. Cada cliente se conectará a un servidor MCP

![mcp architecture](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/mcp-system-architecture.webp)

## FastMCP

Aunque en la documentación de MCP recomiendan instalar `mcp["cli"]`, hay una librería creada por encima llamada `fastmcp`, que ayuda mucho a la hora de crear servidores MCP, así que vamos a usarla

## Crear entorno virtual

Para crear un servidor y un cliente MCP, vamos a crear entornos virtuales con `uv` con las dependencias que vamos a necesitar

### Servidor MCP

Primero creamos una carpeta para el servidor de MCP

In [3]:
!mkdir gitHub_MCP_server

Iniciamos el entorno `uv`

In [7]:
!cd gitHub_MCP_server && uv init .

Initialized project `github-mcp-server` at `/Users/macm1/Documents/web/portafolio/posts/gitHub_MCP_server`


Lo activamos

In [3]:
!cd gitHub_MCP_server && uv venv

Using CPython 3.11.11
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate


E instalamos las linrerías necesarias

In [4]:
!cd gitHub_MCP_server && uv add anthropic fastmcp python-dotenv requests

Resolved 42 packages in 34ms
Installed 40 packages in 71ms
 + annotated-types==0.7.0
 + anyio==4.9.0
 + authlib==1.6.0
 + certifi==2025.6.15
 + cffi==1.17.1
 + charset-normalizer==3.4.2
 + click==8.2.1
 + cryptography==45.0.4
 + distro==1.9.0
 + exceptiongroup==1.3.0
 + fastmcp==2.9.0
 + h11==0.16.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + httpx-sse==0.4.0
 + idna==3.10
 + jiter==0.10.0
 + markdown-it-py==3.0.0
 + mcp==1.9.4
 + mdurl==0.1.2
 + openapi-pydantic==0.5.1
 + pycparser==2.22
 + pydantic==2.11.7
 + pydantic-core==2.33.2
 + pydantic-settings==2.10.0
 + pygments==2.19.2
 + python-dotenv==1.1.1
 + python-multipart==0.0.20
 + requests==2.32.4
 + rich==14.0.0
 + shellingham==1.5.4
 + sniffio==1.3.1
 + sse-starlette==2.3.6
 + starlette==0.47.1
 + typer==0.16.0
 + typing-extensions==4.14.0
 + typing-inspection==0.4.1
 + typing-inspection==0.4.1


### Cliente MCP

Ahora creamos una carpeta donde programaremos el cliente MCP

In [8]:
!mkdir client_MCP

Iniciamos el entorno uv

In [9]:
!cd client_MCP && uv init .

Initialized project `client-mcp` at `/Users/macm1/Documents/web/portafolio/posts/client_MCP`


Lo activamos

In [10]:
!cd client_MCP && uv venv

Using CPython 3.11.11
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate


Y por último, instalamos las librerías necesarias para el cliente.

In [11]:
!cd client_MCP && uv add anthropic fastmcp python-dotenv requests

Resolved 42 packages in 307ms
Prepared 5 packages in 115ms
Installed 40 packages in 117ms
 + annotated-types==0.7.0
 + anthropic==0.55.0
 + anyio==4.9.0
 + authlib==1.6.0
 + certifi==2025.6.15
 + cffi==1.17.1
 + charset-normalizer==3.4.2
 + click==8.2.1
 + cryptography==45.0.4
 + distro==1.9.0
 + exceptiongroup==1.3.0
 + fastmcp==2.9.0
 + h11==0.16.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + httpx-sse==0.4.0
 + idna==3.10
 + jiter==0.10.0
 + markdown-it-py==3.0.0
 + mcp==1.9.4
 + mdurl==0.1.2
 + openapi-pydantic==0.5.1
 + pycparser==2.22
 + pydantic==2.11.7
 + pydantic-core==2.33.2
 + pydantic-settings==2.10.0
 + pygments==2.19.2
 + python-dotenv==1.1.1
 + python-multipart==0.0.20
 + requests==2.32.4
 + rich==14.0.0
 + shellingham==1.5.4
 + sniffio==1.3.1
 + sse-starlette==2.3.6
 + starlette==0.47.1
 + typer==0.16.0
 + typing-extensions==4.14.0
 + typing-inspection==0.4.1
 + typing-inspection==0.4.1


Vamos a usar Sonnet 3.5 como modelo LLM, así que creamos un archivo `.env` en la carpeta del cliente con la API KEY de Claude que se puede obtener en la página [keys](https://console.anthropic.com/settings/keys) de la API de Claude

In [17]:
%%writefile client_MCP/.env

ANTHROPIC_API_KEY="ANTHROPIC_API_KEY"

Writing client_MCP/.env


## MCP básico

Escribímos el mínimo código que necesitamos para tener un servidor MCP

In [2]:
%%writefile gitHub_MCP_server/github_server.py

from mcp.server.fastmcp import FastMCP

# Create an MCP server
mcp = FastMCP("GitHubMCP")


if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

Overwriting gitHub_MCP_server/github_server.py


Como se puede ver, tenemos que crear un objeto `FastMCP` y luego ejecutar el servidor con `mcp.run`.

## Librería con funciones para leer de GitHub

Como vamos a crear un servidor MCP para poder usar utilidades de GitHub, vamos a crear un archivo con las funciones necesarias para construir los headers necesarios para poder usar la API de GitHub.

In [4]:
%%writefile gitHub_MCP_server/github.py

import os
from dotenv import load_dotenv

# Load the GitHub token from the .env file
load_dotenv()
GITHUB_TOKEN = os.environ.get("GITHUB_TOKEN")

# Check if the GitHub token is configured
if not GITHUB_TOKEN:
    print("WARNING: The GITHUB_TOKEN environment variable is not configured.")
    print("Requests to the GitHub API may fail due to rate limits.")
    print("Create a .env file in this directory with GITHUB_TOKEN='your_token_here'")
    raise ValueError("GITHUB_TOKEN is not configured")

# Helper function to create headers for GitHub API requests
def create_github_headers():
    headers = {}
    if GITHUB_TOKEN:
        headers["Authorization"] = f"Bearer {GITHUB_TOKEN}"
    # GitHub recommends including a User-Agent
    headers["User-Agent"] = "MCP_GitHub_Server_Example"
    headers["Accept"] = "application/vnd.github.v3+json" # Good practice
    return headers

Overwriting gitHub_MCP_server/github.py


Para poder construir los headers, necesitamos una token de GitHub. Para ello, vamos a [personal-access-tokens](https://github.com/settings/personal-access-tokens) y creamos una nueva token. Lo copiamos

Ahora, creamos un `.env`, dónde vamos a almacenar el token de GitHub.

In [5]:
%%writefile gitHub_MCP_server/.env

GITHUB_TOKEN = "GITHUB_TOKEN"

Overwriting gitHub_MCP_server/.env


## Crear `tool` de MCP para obtener una lista de issues de un repositorio de GitHub

### Servidor MCP

Añadimos una función para poder listar los issues de un repositorio de GitHub. Para convertir dicha función en una `tool` de MCP, usamos el decorador `@mcp.tool()`

In [6]:
%%writefile gitHub_MCP_server/github_server.py

import httpx
from fastmcp import FastMCP
from github import GITHUB_TOKEN, create_github_headers

# Create a FastMCP server
mcp = FastMCP("GitHubMCP")

@mcp.tool()
async def list_repository_issues(owner: str, repo_name: str) -> list[dict]:
    """
    Lists open issues for a given GitHub repository.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        repo_name: The name of the repository (e.g., 'python-sdk')

    Returns:
        list[dict]: A list of dictionaries, each containing information about an issue
    """
    # Limit to the first 10 issues to avoid long responses
    api_url = f"https://api.github.com/repos/{owner}/{repo_name}/issues?state=open&per_page=10"
    print(f"Fetching issues from {api_url}...")
    
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(api_url, headers=create_github_headers())
            response.raise_for_status()
            issues_data = response.json()
            
            if not issues_data:
                print("No open issues found for this repository.")
                return [{"message": "No open issues found for this repository."}]

            issues_summary = []
            for issue in issues_data:
                # Create a more concise summary
                summary = f"#{issue.get('number', 'N/A')}: {issue.get('title', 'Sin título')}"
                if issue.get('comments', 0) > 0:
                    summary += f" ({issue.get('comments')} comentarios)"
                
                issues_summary.append({
                    "number": issue.get("number"),
                    "title": issue.get("title"),
                    "user": issue.get("user", {}).get("login"),
                    "url": issue.get("html_url"),
                    "comments": issue.get("comments"),
                    "summary": summary
                })
            
            print(f"Found {len(issues_summary)} open issues.")
            
            # Add context information
            result = {
                "total_found": len(issues_summary),
                "repository": f"{owner}/{repo_name}",
                "note": "Mostrando los primeros 10 issues abiertos" if len(issues_summary) == 10 else f"Mostrando todos los {len(issues_summary)} issues abiertos",
                "issues": issues_summary
            }
            
            return [result]
            
        except httpx.HTTPStatusError as e:
            error_message = e.response.json().get("message", "No additional message from API.")
            if e.response.status_code == 403 and GITHUB_TOKEN:
                error_message += " (Rate limit with token or token lacks permissions?)"
            elif e.response.status_code == 403 and not GITHUB_TOKEN:
                error_message += " (Rate limit without token. Consider creating a .env file with GITHUB_TOKEN.)"
            
            print(f"GitHub API error: {e.response.status_code}. {error_message}")
            return [{
                "error": f"GitHub API error: {e.response.status_code}",
                "message": error_message
            }]
        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}")
            return [{"error": f"An unexpected error occurred: {str(e)}"}]


if __name__ == "__main__":
    print("DEBUG: Starting GitHub FastMCP server...")
    print(f"DEBUG: Server name: {mcp.name}")
    print("DEBUG: Available tools: list_repository_issues")
    
    # Initialize and run the server
    mcp.run() 

Overwriting gitHub_MCP_server/github_server.py


### Cliente MCP

Ahora creamos un cliente MCP para poder usar la `tool` que hemos creado

In [7]:
%%writefile client_MCP/client.py

import sys
import asyncio
from contextlib import AsyncExitStack
from anthropic import Anthropic
from dotenv import load_dotenv
import mcp.types as types
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Load environment variables from .env file
load_dotenv()

class FastMCPClient:
    """
    FastMCP client that integrates with Claude to process user queries
    and use tools exposed by a FastMCP server via STDIO.
    """
    
    def __init__(self, server_script_path: str):
        """Initialize the FastMCP client with Anthropic and resource management."""
        self.exit_stack = AsyncExitStack()
        self.anthropic = Anthropic()
        self.session = None
        self.server_params = None
        self.server_script_path = server_script_path
        
    async def connect_to_server(self):
        """
        Connect to the FastMCP server via STDIO.
        """
        print(f"🔗 Connecting to FastMCP server: {self.server_script_path}")
        
        # Determine the server type based on the extension
        if self.server_script_path.endswith('.py'):
            # Python server
            self.server_params = StdioServerParameters(
                command="python",
                args=[self.server_script_path],
                env=None
            )
        elif self.server_script_path.endswith('.js'):
            # JavaScript/Node.js server
            self.server_params = StdioServerParameters(
                command="node", 
                args=[self.server_script_path],
                env=None
            )
        else:
            raise ValueError(f"Unsupported server type. Use .py or .js files. Got: {self.server_script_path}")
        
        # Set up connection to the server
        stdio_transport = await self.exit_stack.enter_async_context(
            stdio_client(self.server_params)
        )
        
        # Create MCP session
        self.session = await self.exit_stack.enter_async_context(
            ClientSession(stdio_transport[0], stdio_transport[1])
        )
        
        # Initialize the session
        await self.session.initialize()
        
        print("✅ Connection established successfully")
        
        # List available tools and resources
        await self.list_available_tools()
        
    async def list_available_tools(self):
        """List available tools in the FastMCP server."""
        try:
            # Get list of tools from the server
            tools_result = await self.session.list_tools()
            
            if tools_result.tools:
                print(f"\n🛠️  Available tools ({len(tools_result.tools)}):")
                print("=" * 50)
                
                for tool in tools_result.tools:
                    print(f"📋 {tool.name}")
                    if tool.description:
                        print(f"   Description: {tool.description}")
                    
                    # Show parameters if available
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        if 'properties' in tool.inputSchema:
                            params = list(tool.inputSchema['properties'].keys())
                            print(f"   Parameters: {', '.join(params)}")
                    print()
            else:
                print("⚠️  No tools found in the server")
                
        except Exception as e:
            print(f"❌ Error listing tools: {str(e)}")

    async def call_tool(self, tool_name: str, arguments: dict) -> str:
        """
        Call a tool on the FastMCP server via STDIO.
        
        Args:
            tool_name: Name of the tool to call
            arguments: Arguments for the tool
            
        Returns:
            str: Result from the tool
        """
        try:
            print(f"🔧 Calling tool: {tool_name}")
            print(f"📝 Arguments: {arguments}")
            
            # Execute tool on the MCP server
            tool_result = await self.session.call_tool(tool_name, arguments)
            
            print(f"✅ Tool executed successfully")
            
            # Format result for Claude
            if tool_result.content:
                # Combine all content into a single result
                combined_content = ""
                for content in tool_result.content:
                    # Check if it's TextContent (has text attribute)
                    if hasattr(content, 'text'):
                        combined_content += content.text + "\n"
                    # Check if it's ImageContent (has data attribute)  
                    elif hasattr(content, 'data'):
                        combined_content += f"[Binary content returned by tool {tool_name}]\n"
                    else:
                        combined_content += f"[Unknown content type returned by tool {tool_name}]\n"
                
                return combined_content.strip()
            else:
                return "Tool executed without response content"
                    
        except Exception as e:
            error_msg = f"❌ Error calling tool {tool_name}: {str(e)}"
            print(error_msg)
            return error_msg
    
    async def process_query(self, query: str) -> str:
        """
        Process a user query, interacting with Claude and FastMCP tools.
        
        Args:
            query: User query
            
        Returns:
            str: Final processed response
        """
        try:
            # Get available tools, resources and templates from the server
            tools_result = await self.session.list_tools()
            
            # Prepare tools for Claude in correct format
            claude_tools = []
            for tool in tools_result.tools:
                claude_tool = {
                    "name": tool.name,
                    "description": tool.description or f"Tool {tool.name}",
                    "input_schema": tool.inputSchema or {"type": "object", "properties": {}}
                }
                claude_tools.append(claude_tool)
            
            # Create initial message for Claude
            messages = [
                {
                    "role": "user",
                    "content": query
                }
            ]
            
            # First call to Claude
            response = self.anthropic.messages.create(
                model="claude-3-5-sonnet-20241022",
                max_tokens=6000,
                messages=messages,
                tools=claude_tools if claude_tools else None
            )
            
            # Process Claude's response
            response_text = ""
            
            for content_block in response.content:
                if content_block.type == "text":
                    response_text += content_block.text
                    
                elif content_block.type == "tool_use":
                    # Claude wants to use a tool
                    tool_name = content_block.name
                    tool_args = content_block.input
                    tool_call_id = content_block.id
                    
                    # Check if it's the special resource reading tool
                    if tool_name == "read_mcp_resource":
                        # Read resource directly
                        tool_result = await self.read_resource(tool_args["uri"])
                    else:
                        # Execute regular tool on the FastMCP server
                        tool_result = await self.call_tool(tool_name, tool_args)
                    
                    # Add tool result to the conversation
                    messages.append({
                        "role": "assistant", 
                        "content": response.content
                    })
                    
                    messages.append({
                        "role": "user",
                        "content": [{
                            "type": "tool_result",
                            "tool_use_id": tool_call_id,
                            "content": f"Tool result: {tool_result}"
                        }]
                    })
                    
                    # Second call to Claude with the tool result
                    final_response = self.anthropic.messages.create(
                        model="claude-3-5-sonnet-20241022",
                        max_tokens=6000,
                        messages=messages,
                        tools=claude_tools if claude_tools else None
                    )
                    
                    # Extract text from the final response
                    for final_content in final_response.content:
                        if final_content.type == "text":
                            response_text += final_content.text
            
            return response_text
            
        except Exception as e:
            error_msg = f"❌ Error processing query: {str(e)}"
            print(error_msg)
            return error_msg
    
    async def chat_loop(self):
        """
        Main chat loop with user interaction.
        """
        print("\n🤖 FastMCP client started. Write 'quit', 'q', 'exit', 'salir' to exit.")
        print("💬 You can ask questions about GitHub repositories!")
        print("📚 The client can use both tools and resources from the FastMCP server")
        print("-" * 60)
        
        while True:
            try:
                # Get user input
                user_input = input("\n👤 You: ").strip()
                
                if user_input.lower() in ['quit', 'q', 'exit', 'salir']:
                    print("👋 Bye!")
                    break
                    
                if not user_input:
                    continue
                
                print("\n🤔 Claude is thinking...")
                
                # Process query
                response = await self.process_query(user_input)
                
                # Show response
                print(f"\n🤖 Claude: {response}")
                
            except KeyboardInterrupt:
                print("\n\n👋 Disconnecting...")
                break
            except Exception as e:
                print(f"\n❌ Error in chat: {str(e)}")
                continue

    async def cleanup(self):
        """Clean up resources and close connections."""
        print("🧹 Cleaning up resources...")
        await self.exit_stack.aclose()
        print("✅ Resources released")


async def main():
    """
    Main function that initializes and runs the FastMCP client.
    """
    # Verify command line arguments
    if len(sys.argv) != 2:
        print("❌ Usage: python client.py <path_to_fastmcp_server>")
        print("📝 Example: python client.py ../MCP_github/github_server.py")
        sys.exit(1)
    
    server_script_path = sys.argv[1]
    
    print(f"🚀 Starting FastMCP client...")
    print(f"📄 Server script: {server_script_path}")
    
    # Create and run client
    client = FastMCPClient(server_script_path)
    
    try:
        # Connect to the server
        await client.connect_to_server()
        
        # Start chat loop
        await client.chat_loop()
        
    except Exception as e:
        print(f"❌ Fatal error: {str(e)}")
        print("💡 Make sure:")
        print("   1. The server script path is correct")
        print("   2. You have ANTHROPIC_API_KEY in your .env file")
        print("   3. The server script is executable")
    finally:
        # Ensure resources are cleaned up
        await client.cleanup()


if __name__ == "__main__":
    # Entry point of the script
    asyncio.run(main()) 

Overwriting client_MCP/client.py


Explicación del cliente MCP

 * En `main` se comprueba que se ha pasado un argumento con el path del servidor MCP.
 * Se crea un objeto de la clase `FastMCPClient` con el path del servidor MCP. Al crear el objeto se ejecuta el método `__init__` que crea la conexión con el LLM de Anthropic, que va a ser el LLM que va a poner el "cerebro"
 * Se intenta conectar con el servidor MCP llamando al método `connect_to_server` que se encarga establecer el método de transporte, abrir una sesión con el servidor MCP y listar las herramientas disponibles.
 * Si se ha podido conectar, se llama al método `chat_loop` que es un bucle infinito para chatear con el LLM que se acaba de crear en el cliente. Solo se para la ejecución cuando se introduce `quit`, `q`, `exit` o `salir` en el chat.
 * Se procesa la entrada del usuario con el método `process_query` que obtiene la lista de `tool`s disponibles y hace una petición al LLM con el mensaje del usuario y la lista de `tool`s
   * Si el LLM responde con texto, se devuelve el texto, que será impreso
   * Si el LLM responde con `tool_use`, se obtiene el nombre de la `tool`, los argumentos y se crea una ID de ejecución. Se ejecuta la tool. Con el resultado de la tool, se crea un nuevo mensaje que se le manda al LLM para que lo procese y genere una respuesta, que será devuelta e impresa.
 * Cuando se termine la conversación, se llamará al método `cleanup`, que cerrará lo que sea necesario cerrar.

### Prueba del cliente y servidor MCP

Nos vamos a la ruta del cliente y lo ejecutamos, dándole la ruta del servidor MCP.

In [9]:
!cd client_MCP && source .venv/bin/activate && python client.py ../gitHub_MCP_server/github_server.py

cd client_MCP && source .venv/bin/activate && python client.py ../gitHub_MCP_server/github_server.py
🚀 Starting FastMCP client...
📄 Server script: ../gitHub_MCP_server/github_server.py
🔗 Connecting to FastMCP server: ../gitHub_MCP_server/github_server.py
[06/28/25 08:50:20] INFO     Starting MCP server 'GitHubMCP' with transport 'stdio'                          server.py:1246
✅ Connection established successfully

🛠️  Available tools (1):
📋 list_repository_issues
   Description: Lists open issues for a given GitHub repository.

Args:
owner: The owner of the repository (e.g., 'modelcontextprotocol')
repo_name: The name of the repository (e.g., 'python-sdk')

Returns:
list[dict]: A list of dictionaries, each containing information about an issue
Parameters: owner, repo_name

🤖 FastMCP client started. Write 'quit', 'q', 'exit', 'salir' to exit.
💬 You can ask questions about GitHub repositories!
📚 The client can use both tools and resources from the FastMCP server
-------------------------

### Crear tool para obtener imágenes

In [10]:
%%writefile gitHub_MCP_server/github_server.py

import httpx
from typing import Optional
from mcp.server.fastmcp import FastMCP, Image
from github import GITHUB_TOKEN, create_github_headers

# Create an MCP server
mcp = FastMCP("GitHubMCP")

@mcp.tool()
async def list_repository_issues(owner: str, repo_name: str) -> list[dict]:
    """
    Lists open issues for a given GitHub repository.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        repo_name: The name of the repository (e.g., 'python-sdk')

    Returns:
        list[dict]: A list of dictionaries, each containing information about an issue
    """
    api_url = f"https://api.github.com/repos/{owner}/{repo_name}/issues?state=open"
    print(f"Fetching issues from {api_url}...")
    
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(api_url, headers=create_github_headers())
            response.raise_for_status()
            issues_data = response.json()
            
            if not issues_data:
                print("No open issues found for this repository.")
                return [{"message": "No open issues found for this repository."}]

            issues_summary = []
            for issue in issues_data:
                issues_summary.append({
                    "title": issue.get("title"),
                    "number": issue.get("number"),
                    "user": issue.get("user", {}).get("login"),
                    "url": issue.get("html_url"),
                    "comments": issue.get("comments")
                })
            
            print(f"Found {len(issues_summary)} open issues.")
            return issues_summary
        except httpx.HTTPStatusError as e:
            error_message = e.response.json().get("message", "No additional message from API.")
            if e.response.status_code == 403 and GITHUB_TOKEN:
                error_message += " (Rate limit with token or token lacks permissions?)"
            elif e.response.status_code == 403 and not GITHUB_TOKEN:
                error_message += " (Rate limit without token. Consider creating a .env file with GITHUB_TOKEN.)"
            
            print(f"GitHub API error: {e.response.status_code}. {error_message}")
            return [{
                "error": f"GitHub API error: {e.response.status_code}",
                "message": error_message
            }]
        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}")
            return [{"error": f"An unexpected error occurred: {str(e)}"}]

@mcp.tool()
async def get_repository_image(owner: str) -> Optional[Image]:
    """
    Get the image of a github profile.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')

    Returns:
        Optional[Image]: The Image object if successful, otherwise None.
    """
    api_url = f"https://api.github.com/users/{owner}"
    print(f"Fetching user data for '{owner}' from {api_url}")

    async with httpx.AsyncClient() as client:
        try:
            # 1. Get user data to find the avatar URL
            user_response = await client.get(api_url, headers=create_github_headers())
            user_response.raise_for_status()
            user_data = user_response.json()

            avatar_url = user_data.get("avatar_url")
            if not avatar_url:
                message = f"Could not find avatar URL for user '{owner}'."
                print(message)
                return None

            # 2. Fetch the image from the avatar URL
            print(f"Fetching image from {avatar_url}")
            image_response = await client.get(avatar_url)
            image_response.raise_for_status()
            image_data = image_response.content

            # 3. Return the image object and a success message
            image = Image(data=image_data, format="png")
            print(f"Successfully retrieved avatar for '{owner}'.")
            return image

        except httpx.HTTPStatusError as e:
            message = f"GitHub API error: {e.response.status_code} - {e.response.text}"
            print(f"Failed to get repository image: {message}")
            return None
        except Exception as e:
            message = f"An unexpected error occurred: {str(e)}"
            print(f"Failed to get repository image: {message}")
            return None


if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

Overwriting gitHub_MCP_server/github_server.py


### Context

In [11]:
%%writefile gitHub_MCP_server/github_server.py

import httpx
from typing import Optional
from mcp.server.fastmcp import FastMCP, Image, Context
from github import GITHUB_TOKEN, create_github_headers

# Create an MCP server
mcp = FastMCP("GitHubMCP")

@mcp.tool()
async def list_repository_issues(owner: str, repo_name: str, ctx: Context) -> list[dict]:
    """
    Lists open issues for a given GitHub repository.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        repo_name: The name of the repository (e.g., 'python-sdk')

    Returns:
        list[dict]: A list of dictionaries, each containing information about an issue
    """
    api_url = f"https://api.github.com/repos/{owner}/{repo_name}/issues?state=open"
    ctx.info(f"Fetching issues from {api_url}...")
    
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(api_url, headers=create_github_headers())
            response.raise_for_status()
            issues_data = response.json()
            
            if not issues_data:
                ctx.info("No open issues found for this repository.")
                return [{"message": "No open issues found for this repository."}]

            issues_summary = []
            for issue in issues_data:
                issues_summary.append({
                    "title": issue.get("title"),
                    "number": issue.get("number"),
                    "user": issue.get("user", {}).get("login"),
                    "url": issue.get("html_url"),
                    "comments": issue.get("comments")
                })
            
            ctx.info(f"Found {len(issues_summary)} open issues.")
            return issues_summary
        except httpx.HTTPStatusError as e:
            error_message = e.response.json().get("message", "No additional message from API.")
            if e.response.status_code == 403 and GITHUB_TOKEN:
                error_message += " (Rate limit with token or token lacks permissions?)"
            elif e.response.status_code == 403 and not GITHUB_TOKEN:
                error_message += " (Rate limit without token. Consider creating a .env file with GITHUB_TOKEN.)"
            
            ctx.info(f"GitHub API error: {e.response.status_code}. {error_message}")
            return [{
                "error": f"GitHub API error: {e.response.status_code}",
                "message": error_message
            }]
        except Exception as e:
            ctx.info(f"An unexpected error occurred: {str(e)}")
            return [{"error": f"An unexpected error occurred: {str(e)}"}]

@mcp.tool()
async def get_repository_image(owner: str, ctx: Context) -> Optional[Image]:
    """
    Get the image of a github profile.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        ctx: The MCP context for logging.

    Returns:
        Optional[Image]: The Image object if successful, otherwise None.
    """
    api_url = f"https://api.github.com/users/{owner}"
    ctx.info(f"Fetching user data for '{owner}' from {api_url}")

    async with httpx.AsyncClient() as client:
        try:
            # 1. Get user data to find the avatar URL
            user_response = await client.get(api_url, headers=create_github_headers())
            user_response.raise_for_status()
            user_data = user_response.json()

            avatar_url = user_data.get("avatar_url")
            if not avatar_url:
                message = f"Could not find avatar URL for user '{owner}'."
                ctx.info(message)
                return None

            # 2. Fetch the image from the avatar URL
            ctx.info(f"Fetching image from {avatar_url}")
            image_response = await client.get(avatar_url)
            image_response.raise_for_status()
            image_data = image_response.content

            # 3. Return the image object and a success message
            image = Image(data=image_data, format="png")
            ctx.info(f"Successfully retrieved avatar for '{owner}'.")
            return image

        except httpx.HTTPStatusError as e:
            message = f"GitHub API error: {e.response.status_code} - {e.response.text}"
            ctx.info(f"Failed to get repository image: {message}")
            return None
        except Exception as e:
            message = f"An unexpected error occurred: {str(e)}"
            ctx.info(f"Failed to get repository image: {message}")
            return None


if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

Overwriting gitHub_MCP_server/github_server.py


### Resource

In [5]:
%%writefile gitHub_MCP_server/github_server.py

import httpx
from typing import Optional
from mcp.server.fastmcp import FastMCP, Image, Context
from github import GITHUB_TOKEN, create_github_headers

# Create an MCP server
mcp = FastMCP("GitHubMCP")

@mcp.resource("github://repo/{owner}/{repo_name}")
async def get_repository_info(owner: str, repo_name: str) -> dict:
    """
    Gets detailed information for a given GitHub repository.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        repo_name: The name of the repository (e.g., 'python-sdk')

    Returns:
        dict: A dictionary containing repository details.
    """
    api_url = f"https://api.github.com/repos/{owner}/{repo_name}"
    print(f"Fetching repository info from {api_url}...")
    
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(api_url, headers=create_github_headers())
            response.raise_for_status()
            repo_data = response.json()
            
            return {
                "full_name": repo_data.get("full_name"),
                "description": repo_data.get("description"),
                "stars": repo_data.get("stargazers_count"),
                "forks": repo_data.get("forks_count"),
                "open_issues": repo_data.get("open_issues_count"),
                "language": repo_data.get("language"),
                "url": repo_data.get("html_url")
            }
        except httpx.HTTPStatusError as e:
            error_message = e.response.json().get("message", "No additional message from API.")
            if e.response.status_code == 403 and GITHUB_TOKEN:
                error_message += " (Rate limit with token or token lacks permissions?)"
            elif e.response.status_code == 403 and not GITHUB_TOKEN:
                error_message += " (Rate limit without token. Consider creating a .env file with GITHUB_TOKEN.)"
            
            print(f"GitHub API error: {e.response.status_code}. {error_message}")
            return {
                "error": f"GitHub API error: {e.response.status_code}",
                "message": error_message
            }
        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}")
            return {"error": f"An unexpected error occurred: {str(e)}"}

@mcp.tool()
async def list_repository_issues(owner: str, repo_name: str, ctx: Context) -> list[dict]:
    """
    Lists open issues for a given GitHub repository.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        repo_name: The name of the repository (e.g., 'python-sdk')
        ctx: The MCP context for logging.

    Returns:
        list[dict]: A list of dictionaries, each containing information about an issue
    """
    api_url = f"https://api.github.com/repos/{owner}/{repo_name}/issues?state=open"
    ctx.info(f"Fetching issues from {api_url}...")
    
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(api_url, headers=create_github_headers())
            response.raise_for_status()
            issues_data = response.json()
            
            if not issues_data:
                ctx.info("No open issues found for this repository.")
                return [{"message": "No open issues found for this repository."}]

            issues_summary = []
            for issue in issues_data:
                issues_summary.append({
                    "title": issue.get("title"),
                    "number": issue.get("number"),
                    "user": issue.get("user", {}).get("login"),
                    "url": issue.get("html_url"),
                    "comments": issue.get("comments")
                })
            
            ctx.info(f"Found {len(issues_summary)} open issues.")
            return issues_summary
        except httpx.HTTPStatusError as e:
            error_message = e.response.json().get("message", "No additional message from API.")
            if e.response.status_code == 403 and GITHUB_TOKEN:
                error_message += " (Rate limit with token or token lacks permissions?)"
            elif e.response.status_code == 403 and not GITHUB_TOKEN:
                error_message += " (Rate limit without token. Consider creating a .env file with GITHUB_TOKEN.)"
            
            ctx.info(f"GitHub API error: {e.response.status_code}. {error_message}")
            return [{
                "error": f"GitHub API error: {e.response.status_code}",
                "message": error_message
            }]
        except Exception as e:
            ctx.info(f"An unexpected error occurred: {str(e)}")
            return [{"error": f"An unexpected error occurred: {str(e)}"}]

@mcp.tool()
async def get_repository_image(owner: str, ctx: Context) -> Optional[Image]:
    """
    Get the image of a github profile.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        ctx: The MCP context for logging.

    Returns:
        Optional[Image]: The Image object if successful, otherwise None.
    """
    api_url = f"https://api.github.com/users/{owner}"
    ctx.info(f"Fetching user data for '{owner}' from {api_url}")

    async with httpx.AsyncClient() as client:
        try:
            # 1. Get user data to find the avatar URL
            user_response = await client.get(api_url, headers=create_github_headers())
            user_response.raise_for_status()
            user_data = user_response.json()

            avatar_url = user_data.get("avatar_url")
            if not avatar_url:
                message = f"Could not find avatar URL for user '{owner}'."
                ctx.info(message)
                return None

            # 2. Fetch the image from the avatar URL
            ctx.info(f"Fetching image from {avatar_url}")
            image_response = await client.get(avatar_url)
            image_response.raise_for_status()
            image_data = image_response.content

            # 3. Return the image object and a success message
            image = Image(data=image_data, format="png")
            ctx.info(f"Successfully retrieved avatar for '{owner}'.")
            return image

        except httpx.HTTPStatusError as e:
            message = f"GitHub API error: {e.response.status_code} - {e.response.text}"
            ctx.info(f"Failed to get repository image: {message}")
            return None
        except Exception as e:
            message = f"An unexpected error occurred: {str(e)}"
            ctx.info(f"Failed to get repository image: {message}")
            return None


if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

Overwriting gitHub_MCP_server/github_server.py


### Crear tool para obtener un resumen de issues de un repositorio de github

In [7]:
%%writefile gitHub_MCP_server/github_server.py

import httpx
from typing import Optional
from mcp.server.fastmcp import FastMCP, Image, Context
from github import GITHUB_TOKEN, create_github_headers
import anyio

# Create an MCP server
mcp = FastMCP("GitHubMCP")

@mcp.resource("github://repo/{owner}/{repo_name}")
async def get_repository_info(owner: str, repo_name: str) -> dict:
    """
    Gets detailed information for a given GitHub repository.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        repo_name: The name of the repository (e.g., 'python-sdk')

    Returns:
        dict: A dictionary containing repository details.
    """
    api_url = f"https://api.github.com/repos/{owner}/{repo_name}"
    print(f"Fetching repository info from {api_url}...")
    
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(api_url, headers=create_github_headers())
            response.raise_for_status()
            repo_data = response.json()
            
            return {
                "full_name": repo_data.get("full_name"),
                "description": repo_data.get("description"),
                "stars": repo_data.get("stargazers_count"),
                "forks": repo_data.get("forks_count"),
                "open_issues": repo_data.get("open_issues_count"),
                "language": repo_data.get("language"),
                "url": repo_data.get("html_url")
            }
        except httpx.HTTPStatusError as e:
            error_message = e.response.json().get("message", "No additional message from API.")
            if e.response.status_code == 403 and GITHUB_TOKEN:
                error_message += " (Rate limit with token or token lacks permissions?)"
            elif e.response.status_code == 403 and not GITHUB_TOKEN:
                error_message += " (Rate limit without token. Consider creating a .env file with GITHUB_TOKEN.)"
            
            print(f"GitHub API error: {e.response.status_code}. {error_message}")
            return {
                "error": f"GitHub API error: {e.response.status_code}",
                "message": error_message
            }
        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}")
            return {"error": f"An unexpected error occurred: {str(e)}"}

@mcp.tool()
async def list_repository_issues(owner: str, repo_name: str, ctx: Context) -> list[dict]:
    """
    Lists open issues for a given GitHub repository.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        repo_name: The name of the repository (e.g., 'python-sdk')
        ctx: The MCP context for logging.

    Returns:
        list[dict]: A list of dictionaries, each containing information about an issue
    """
    api_url = f"https://api.github.com/repos/{owner}/{repo_name}/issues?state=open"
    ctx.info(f"Fetching issues from {api_url}...")
    
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(api_url, headers=create_github_headers())
            response.raise_for_status()
            issues_data = response.json()
            
            if not issues_data:
                ctx.info("No open issues found for this repository.")
                return [{"message": "No open issues found for this repository."}]

            issues_summary = []
            for issue in issues_data:
                issues_summary.append({
                    "title": issue.get("title"),
                    "number": issue.get("number"),
                    "user": issue.get("user", {}).get("login"),
                    "url": issue.get("html_url"),
                    "comments": issue.get("comments")
                })
            
            ctx.info(f"Found {len(issues_summary)} open issues.")
            return issues_summary
        except httpx.HTTPStatusError as e:
            error_message = e.response.json().get("message", "No additional message from API.")
            if e.response.status_code == 403 and GITHUB_TOKEN:
                error_message += " (Rate limit with token or token lacks permissions?)"
            elif e.response.status_code == 403 and not GITHUB_TOKEN:
                error_message += " (Rate limit without token. Consider creating a .env file with GITHUB_TOKEN.)"
            
            ctx.info(f"GitHub API error: {e.response.status_code}. {error_message}")
            return [{
                "error": f"GitHub API error: {e.response.status_code}",
                "message": error_message
            }]
        except Exception as e:
            ctx.info(f"An unexpected error occurred: {str(e)}")
            return [{"error": f"An unexpected error occurred: {str(e)}"}]

@mcp.tool()
async def get_repository_image(owner: str, ctx: Context) -> Optional[Image]:
    """
    Get the image of a github profile.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        ctx: The MCP context for logging.

    Returns:
        Optional[Image]: The Image object if successful, otherwise None.
    """
    api_url = f"https://api.github.com/users/{owner}"
    ctx.info(f"Fetching user data for '{owner}' from {api_url}")

    async with httpx.AsyncClient() as client:
        try:
            # 1. Get user data to find the avatar URL
            user_response = await client.get(api_url, headers=create_github_headers())
            user_response.raise_for_status()
            user_data = user_response.json()

            avatar_url = user_data.get("avatar_url")
            if not avatar_url:
                message = f"Could not find avatar URL for user '{owner}'."
                ctx.info(message)
                return None

            # 2. Fetch the image from the avatar URL
            ctx.info(f"Fetching image from {avatar_url}")
            image_response = await client.get(avatar_url)
            image_response.raise_for_status()
            image_data = image_response.content

            # 3. Return the image object and a success message
            image = Image(data=image_data, format="png")
            ctx.info(f"Successfully retrieved avatar for '{owner}'.")
            return image

        except httpx.HTTPStatusError as e:
            message = f"GitHub API error: {e.response.status_code} - {e.response.text}"
            ctx.info(f"Failed to get repository image: {message}")
            return None
        except Exception as e:
            message = f"An unexpected error occurred: {str(e)}"
            ctx.info(f"Failed to get repository image: {message}")
            return None

@mcp.tool()
async def analyze_repository_activity(owner: str, repo_name: str, ctx: Context) -> dict:
    """
    Analyze repository activity with detailed progress reporting.
    
    This function demonstrates Context capabilities while performing useful analysis.
    """
    ctx.info("Starting repository activity analysis...")
    total_steps = 5
    
    # Step 1: Get repository info
    await ctx.report_progress(1, total_steps, "Fetching repository information...")
    repo_info = await get_repository_info(owner, repo_name)
    
    # Step 2: Get recent issues
    await ctx.report_progress(2, total_steps, "Analyzing recent issues...")
    issues = await list_repository_issues(owner, repo_name, ctx)
    
    # Step 3: Analyze issue patterns
    await ctx.report_progress(3, total_steps, "Detecting patterns in issues...")
    # Simular análisis complejo con delay
    await anyio.sleep(1)
    
    # Step 4: Calculate metrics
    await ctx.report_progress(4, total_steps, "Calculating activity metrics...")
    metrics = {
        "total_issues": len(issues) if isinstance(issues, list) else 0,
        "avg_comments": sum(issue.get('comments', 0) for issue in issues if isinstance(issue, dict)) / len(issues) if issues else 0,
        "active_contributors": len(set(issue.get('user') for issue in issues if isinstance(issue, dict)))
    }
    
    # Step 5: Generate report
    await ctx.report_progress(5, total_steps, "Generating final report...")
    
    result = {
        "repository": f"{owner}/{repo_name}",
        "analysis_timestamp": "2024-01-01",  # Podrías usar datetime.now()
        "metrics": metrics,
        "summary": f"Repository has {metrics['total_issues']} recent issues with {metrics['active_contributors']} contributors",
        "repo_info": repo_info,
    }
    
    ctx.info("Repository activity analysis completed successfully!")
    return result


if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

Overwriting gitHub_MCP_server/github_server.py


### Prompt

In [8]:
%%writefile gitHub_MCP_server/github_server.py

import httpx
from typing import Optional
from mcp.server.fastmcp import FastMCP, Image, Context
from github import GITHUB_TOKEN, create_github_headers
import anyio

# Create an MCP server
mcp = FastMCP("GitHubMCP")

@mcp.prompt()
def summarize_repository_issues(owner: str, repo_name: str) -> str:
    """
    Generates a prompt to summarize the open issues of a repository.

    Args:
        owner: The owner of the repository.
        repo_name: The name of the repository.

    Returns:
        A string with the request for the LLM.
    """
    return f"Please provide a summary of the open issues for the repository '{owner}/{repo_name}'."

@mcp.resource("github://repo/{owner}/{repo_name}")
async def get_repository_info(owner: str, repo_name: str) -> dict:
    """
    Gets detailed information for a given GitHub repository.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        repo_name: The name of the repository (e.g., 'python-sdk')

    Returns:
        dict: A dictionary containing repository details.
    """
    api_url = f"https://api.github.com/repos/{owner}/{repo_name}"
    print(f"Fetching repository info from {api_url}...")
    
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(api_url, headers=create_github_headers())
            response.raise_for_status()
            repo_data = response.json()
            
            return {
                "full_name": repo_data.get("full_name"),
                "description": repo_data.get("description"),
                "stars": repo_data.get("stargazers_count"),
                "forks": repo_data.get("forks_count"),
                "open_issues": repo_data.get("open_issues_count"),
                "language": repo_data.get("language"),
                "url": repo_data.get("html_url")
            }
        except httpx.HTTPStatusError as e:
            error_message = e.response.json().get("message", "No additional message from API.")
            if e.response.status_code == 403 and GITHUB_TOKEN:
                error_message += " (Rate limit with token or token lacks permissions?)"
            elif e.response.status_code == 403 and not GITHUB_TOKEN:
                error_message += " (Rate limit without token. Consider creating a .env file with GITHUB_TOKEN.)"
            
            print(f"GitHub API error: {e.response.status_code}. {error_message}")
            return {
                "error": f"GitHub API error: {e.response.status_code}",
                "message": error_message
            }
        except Exception as e:
            print(f"An unexpected error occurred: {str(e)}")
            return {"error": f"An unexpected error occurred: {str(e)}"}

@mcp.tool()
async def list_repository_issues(owner: str, repo_name: str, ctx: Context) -> list[dict]:
    """
    Lists open issues for a given GitHub repository.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        repo_name: The name of the repository (e.g., 'python-sdk')
        ctx: The MCP context for logging.

    Returns:
        list[dict]: A list of dictionaries, each containing information about an issue
    """
    api_url = f"https://api.github.com/repos/{owner}/{repo_name}/issues?state=open"
    ctx.info(f"Fetching issues from {api_url}...")
    
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(api_url, headers=create_github_headers())
            response.raise_for_status()
            issues_data = response.json()
            
            if not issues_data:
                ctx.info("No open issues found for this repository.")
                return [{"message": "No open issues found for this repository."}]

            issues_summary = []
            for issue in issues_data:
                issues_summary.append({
                    "title": issue.get("title"),
                    "number": issue.get("number"),
                    "user": issue.get("user", {}).get("login"),
                    "url": issue.get("html_url"),
                    "comments": issue.get("comments")
                })
            
            ctx.info(f"Found {len(issues_summary)} open issues.")
            return issues_summary
        except httpx.HTTPStatusError as e:
            error_message = e.response.json().get("message", "No additional message from API.")
            if e.response.status_code == 403 and GITHUB_TOKEN:
                error_message += " (Rate limit with token or token lacks permissions?)"
            elif e.response.status_code == 403 and not GITHUB_TOKEN:
                error_message += " (Rate limit without token. Consider creating a .env file with GITHUB_TOKEN.)"
            
            ctx.info(f"GitHub API error: {e.response.status_code}. {error_message}")
            return [{
                "error": f"GitHub API error: {e.response.status_code}",
                "message": error_message
            }]
        except Exception as e:
            ctx.info(f"An unexpected error occurred: {str(e)}")
            return [{"error": f"An unexpected error occurred: {str(e)}"}]

@mcp.tool()
async def get_repository_image(owner: str, ctx: Context) -> Optional[Image]:
    """
    Get the image of a github profile.

    Args:
        owner: The owner of the repository (e.g., 'modelcontextprotocol')
        ctx: The MCP context for logging.

    Returns:
        Optional[Image]: The Image object if successful, otherwise None.
    """
    api_url = f"https://api.github.com/users/{owner}"
    ctx.info(f"Fetching user data for '{owner}' from {api_url}")

    async with httpx.AsyncClient() as client:
        try:
            # 1. Get user data to find the avatar URL
            user_response = await client.get(api_url, headers=create_github_headers())
            user_response.raise_for_status()
            user_data = user_response.json()

            avatar_url = user_data.get("avatar_url")
            if not avatar_url:
                message = f"Could not find avatar URL for user '{owner}'."
                ctx.info(message)
                return None

            # 2. Fetch the image from the avatar URL
            ctx.info(f"Fetching image from {avatar_url}")
            image_response = await client.get(avatar_url)
            image_response.raise_for_status()
            image_data = image_response.content

            # 3. Return the image object and a success message
            image = Image(data=image_data, format="png")
            ctx.info(f"Successfully retrieved avatar for '{owner}'.")
            return image

        except httpx.HTTPStatusError as e:
            message = f"GitHub API error: {e.response.status_code} - {e.response.text}"
            ctx.info(f"Failed to get repository image: {message}")
            return None
        except Exception as e:
            message = f"An unexpected error occurred: {str(e)}"
            ctx.info(f"Failed to get repository image: {message}")
            return None

@mcp.tool()
async def issues_summary(issues: list[dict], ctx: Context) -> str:
    """
    Create a formatted summary of the provided issues for easy consumption by an LLM.

    Args:
        issues: The list of issues (as dictionaries) to summarize.
        ctx: The MCP context, used for logging.

    Returns:
        str: A formatted text summary of the issues that can be easily processed by an LLM.
    """
    if not issues or (len(issues) == 1 and "error" in issues[0]):
        return "No issues provided or an error occurred in a previous step."

    ctx.info(f"Formatting {len(issues)} issues for summary...")
    
    # Format the issues in a structured, readable way
    formatted_lines = []
    formatted_lines.append(f"GitHub Repository Issues Summary ({len(issues)} issues)")
    formatted_lines.append("=" * 60)
    formatted_lines.append("")
    
    for i, issue in enumerate(issues, 1):
        title = issue.get('title', 'No Title')
        number = issue.get('number', 'N/A')
        user = issue.get('user', 'N/A')
        url = issue.get('url', 'N/A')
        comments = issue.get('comments', 0)
        
        formatted_lines.append(f"{i}. Issue #{number}: {title}")
        formatted_lines.append(f"   Created by: {user}")
        formatted_lines.append(f"   Comments: {comments}")
        formatted_lines.append(f"   URL: {url}")
        formatted_lines.append("")
        
        # Report progress as we process each issue
        await ctx.report_progress(i, len(issues))

    formatted_lines.append("=" * 60)
    formatted_lines.append("Summary complete. This data is ready for LLM analysis.")
    
    formatted_text = '\n'.join(formatted_lines)
    
    ctx.info("Successfully formatted issues summary.")
    return formatted_text

@mcp.tool()
async def analyze_repository_activity(owner: str, repo_name: str, ctx: Context) -> dict:
    """
    Analyze repository activity with detailed progress reporting.
    
    This function demonstrates Context capabilities while performing useful analysis.
    """
    ctx.info("Starting repository activity analysis...")
    total_steps = 5
    
    # Step 1: Get repository info
    await ctx.report_progress(1, total_steps, "Fetching repository information...")
    repo_info = await get_repository_info(owner, repo_name)
    
    # Step 2: Get recent issues
    await ctx.report_progress(2, total_steps, "Analyzing recent issues...")
    issues = await list_repository_issues(owner, repo_name, ctx)
    
    # Step 3: Analyze issue patterns
    await ctx.report_progress(3, total_steps, "Detecting patterns in issues...")
    # Simular análisis complejo con delay
    await anyio.sleep(1)
    
    # Step 4: Calculate metrics
    await ctx.report_progress(4, total_steps, "Calculating activity metrics...")
    metrics = {
        "total_issues": len(issues) if isinstance(issues, list) else 0,
        "avg_comments": sum(issue.get('comments', 0) for issue in issues if isinstance(issue, dict)) / len(issues) if issues else 0,
        "active_contributors": len(set(issue.get('user') for issue in issues if isinstance(issue, dict)))
    }
    
    # Step 5: Generate report
    await ctx.report_progress(5, total_steps, "Generating final report...")
    
    result = {
        "repository": f"{owner}/{repo_name}",
        "analysis_timestamp": "2024-01-01",  # Podrías usar datetime.now()
        "metrics": metrics,
        "summary": f"Repository has {metrics['total_issues']} recent issues with {metrics['active_contributors']} contributors",
        "repo_info": repo_info,
    }
    
    ctx.info("Repository activity analysis completed successfully!")
    return result


if __name__ == "__main__":
    # Initialize and run the server
    mcp.run(transport='stdio')

Overwriting gitHub_MCP_server/github_server.py


In [ ]:
# Verificar el estado actual de uv
!uv --version


In [ ]:
# Diagnóstico 1: ¿Funciona uv desde el directorio gitHub_MCP_server?
!cd gitHub_MCP_server && uv --version


In [ ]:
# Diagnóstico 2: ¿Dónde está uv?
!cd gitHub_MCP_server && which uv


In [ ]:
# Diagnóstico 3: Comparar PATH en ambos contextos
import os
print("PATH actual en Jupyter:")
print(os.environ.get('PATH'))


In [ ]:
# Opción 1: Instalar uv usando pip (más fácil)
%pip install uv


In [ ]:
# Verificar que uv ahora funciona
!uv --version


In [ ]:
# Diagnóstico 4: PATH desde el directorio de trabajo
!cd gitHub_MCP_server && echo $PATH


In [ ]:
# Versión corregida de los comandos que fallaron:
print("Inicializando proyecto uv...")
!cd gitHub_MCP_server && uv init --quiet . 2>/dev/null || echo "Proyecto ya inicializado"

print("\\nCreando entorno virtual...")
!cd gitHub_MCP_server && uv venv --quiet 2>/dev/null || echo "Entorno virtual ya existe"

print("\\nVerificando versión de uv...")
!cd gitHub_MCP_server && uv --version
